<a href="https://colab.research.google.com/github/sakrbn/11th-Group/blob/main/Speech_embeddings_using_Wav2Vec_in_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Transformers](https://huggingface.co/docs/transformers/en/index)

In [1]:
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2Model

In [2]:
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [3]:
audio_path = "/NLP_sample_audio.wav"

In [4]:
audio_input, sample_rate = torchaudio.load(audio_path)

In [5]:
audio_input

tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.0002, -0.0002, -0.0002],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [6]:
sample_rate

48000

In [7]:
input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt")

ValueError: The model corresponding to this feature extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}
 was trained using a sampling rate of 16000. Please make sure that the provided `raw_speech` input was sampled with 16000 and not 48000.

In [8]:
resample_speech = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(audio_input)

In [9]:
input_values = processor(resample_speech.squeeze().numpy(), return_tensors="pt")

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [10]:
with torch.no_grad():
    output = model(**input_values)

In [11]:
output.last_hidden_state.shape

torch.Size([2, 600, 768])

In [12]:
output.last_hidden_state

tensor([[[-2.8380e-02,  4.2565e-02, -4.5212e-02,  ..., -2.9012e-01,
           4.3971e-02, -4.4809e-03],
         [-2.3099e-02,  4.0491e-02, -4.3261e-02,  ..., -2.8769e-01,
           4.5696e-02, -6.3315e-03],
         [-2.4047e-02,  4.0932e-02, -4.1534e-02,  ..., -2.8880e-01,
           4.6684e-02, -6.1642e-03],
         ...,
         [-3.2517e-02,  6.1847e-02, -6.4664e-02,  ..., -2.9829e-01,
           4.8773e-02,  1.7158e-03],
         [-3.2678e-02,  7.4256e-02, -8.5565e-02,  ..., -3.1184e-01,
           4.1724e-02,  8.8487e-03],
         [-2.8644e-02,  4.6880e-02, -3.6453e-02,  ..., -2.7865e-01,
           4.8742e-02,  8.7679e-03]],

        [[-1.0796e-01,  1.3843e-02,  2.7777e-02,  ...,  8.2587e-05,
           3.8990e-03,  2.1548e-02],
         [-1.0028e-01,  1.7617e-02,  3.3554e-02,  ...,  1.6835e-04,
          -1.8821e-02,  3.1275e-03],
         [-1.1116e-01,  2.0320e-02,  2.7726e-02,  ...,  4.5858e-03,
          -1.3077e-02,  2.1491e-03],
         ...,
         [-9.0106e-02,  2